In [1]:
import os
import sys
import time
import yaml
import shutil
import argparse
import tarfile
import random
import tempfile
import subprocess

import torch
import pytorch_lightning as ptl
from pytorch_lightning import Trainer

import boda


In [2]:
print(torch.__version__)
print(ptl.__version__)

1.7.1
1.3.0


In [3]:
def main(args):
    data_module = getattr(boda.data, args['Main args'].data_module)
    model_module= getattr(boda.model, args['Main args'].model_module)
    graph_module= getattr(boda.graph, args['Main args'].graph_module)

    data = data_module(**vars(data_module.process_args(args)))
    model= model_module(**vars(model_module.process_args(args)))

    model.__class__ = type(
        'BODA_module',
        (model_module,graph_module),
        vars(graph_module.process_args(args))
    )

    trainer = Trainer.from_argparse_args(args['pl.Trainer'])
    os.makedirs(trainer.default_root_dir, exist_ok=True)
    
    trainer.fit(model, data)
    
    #_save_model(data_module, model_module, graph_module, 
    #            model, trainer, args)
    
    return data_module, model_module, graph_module, model, trainer, args

In [4]:
def _save_model(data_module, model_module, graph_module, 
                model, trainer, args):
    local_dir = args['pl.Trainer'].default_root_dir
    save_dict = {
        'data_module'  : data_module.__name__,
        'data_hparams' : data_module.process_args(args),
        'model_module' : model_module.__name__,
        'model_hparams': model_module.process_args(args),
        'graph_module' : graph_module.__name__,
        'graph_hparams': graph_module.process_args(args),
        'model_state_dict': model.state_dict(),
        'timestamp'    : time.strftime("%Y%m%d_%H%M%S"),
        'random_tag'   : random.randint(100000,999999)
    }
    torch.save(save_dict, os.path.join(local_dir,'torch_checkpoint.pt'))
    
    filename=f'model_artifacts__{save_dict["timestamp"]}__{save_dict["random_tag"]}.tar.gz'
    with tempfile.TemporaryDirectory() as tmpdirname:
        tmpdirname = '/tmp/output'
        with tarfile.open(os.path.join(tmpdirname,filename), 'w:gz') as tar:
            tar.add(local_dir,arcname='artifacts')

        if 'gs://' in args['Main args'].artifact_path:
            clound_target = os.path.join(args['Main args'].artifact_path,filename)
            subprocess.check_call(
                ['gsutil', 'cp', os.path.join(tmpdirname,filename), clound_target]
            )
        else:
            os.makedirs(args['Main args'].artifact_path, exist_ok=True)
            shutil.copy(os.path.join(tmpdirname,filename), args['Main args'].artifact_path)


In [5]:
def model_fn(model_dir):
    checkpoint = torch.load(os.path.join(model_dir,'torch_checkpoint.pt'))
    model_module = getattr(boda, checkpoint['model_module'])
    model        = model_module(**checkpoint['model_hparams'])
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f'Loaded model from {checkpoint["timestamp"]}')
    return model


# Process runtime arguments

## Command line args to use for testing

In [6]:
cmd_str = '--data_module BODA2_DataModule ' +\
            '--dataFile_path  /home/ubuntu/boda2/data/BODA.MPRA.txt ' +\
            '--ValSize_pct  5 --TestSize_pct  5 ' +\
            '--batchSize  4096 --paddedSeqLen 600 --numWorkers 1 ' +\
          '--model_module Basset ' +\
            '--n_outputs  3 --loss_criterion MSELoss ' +\
          '--graph_module CNNTransferLearning ' +\
            '--optimizer AdamW ' +\
            '--parent_weights /home/ubuntu/boda2/artifacts/my-model.epoch_5-step_19885.pkl ' +\
            '--frozen_epochs 120 ' +\
          '--gpus 1 --min_epochs 140 --max_epochs 140 --precision 16 --default_root_dir /tmp/output/artifacts ' +\
          '--artifact_path gs://haddath/sgosai/deposit_test '

'python /home/ubuntu/boda2/src/main.py ' + cmd_str

'python /home/ubuntu/boda2/src/main.py --data_module BODA2_DataModule --dataFile_path  ../../data/BODA.MPRA.txt --ValSize_pct  5 --TestSize_pct  5 --batchSize  4096 --paddedSeqLen 600 --numWorkers 1 --model_module Basset --n_outputs  3 --loss_criterion MSELoss --graph_module CNNTransferLearning --optimizer AdamW --parent_weights /home/ubuntu/boda2/artifacts/my-model.epoch_5-step_19885.pkl --frozen_epochs 120 --gpus 1 --min_epochs 140 --max_epochs 140 --precision 16 --default_root_dir /tmp/output/artifacts --artifact_path gs://haddath/sgosai/deposit_test '

## Set base args for script

Basic arguments to identify which submodules are used and where data will be saved

In [7]:
parser = argparse.ArgumentParser(description="BODA trainer", add_help=False)
group = parser.add_argument_group('Main args')
group.add_argument('--data_module', type=str, required=True, help='BODA data module to process dataset.')
group.add_argument('--model_module',type=str, required=True, help='BODA model module to fit dataset.')
group.add_argument('--graph_module',type=str, required=True, help='BODA graph module to define computations.')
group.add_argument('--artifact_path', type=str, default='/opt/ml/checkpoints/', help='Path where model artifacts are deposited.')
group.add_argument('--pretrained_weights', type=str, help='Pretrained weights.')



_StoreAction(option_strings=['--pretrained_weights'], dest='pretrained_weights', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='Pretrained weights.', metavar=None)

In [8]:
known_args, leftover_args = parser.parse_known_args(
    cmd_str.rstrip().split()
)

In [9]:
known_args

Namespace(artifact_path='gs://haddath/sgosai/deposit_test', data_module='BODA2_DataModule', graph_module='CNNTransferLearning', model_module='Basset', pretrained_weights=None)

## Extract first-order submodule args

Get submodule specific arguments.

In [10]:
Data  = getattr(boda.data,  known_args.data_module)
Model = getattr(boda.model, known_args.model_module)
Graph = getattr(boda.graph, known_args.graph_module)

parser = Data.add_data_specific_args(parser)
parser = Model.add_model_specific_args(parser)
parser = Graph.add_graph_specific_args(parser)

known_args, leftover_args = parser.parse_known_args(
    cmd_str.rstrip().split()
)


In [11]:
known_args

Namespace(TestSize_pct=5.0, ValSize_pct=5.0, activityColumns=['K562', 'HepG2', 'SKNSH'], artifact_path='gs://haddath/sgosai/deposit_test', batchSize=4096, conv1_channels=300, conv1_kernel_size=19, conv2_channels=200, conv2_kernel_size=11, conv3_channels=200, conv3_kernel_size=7, dataFile_path='../../data/BODA.MPRA.txt', data_module='BODA2_DataModule', dropout_p=0.3, frozen_epochs=120, graph_module='CNNTransferLearning', linear1_channels=1000, linear2_channels=1000, loss_criterion='MSELoss', model_module='Basset', n_outputs=3, numWorkers=1, optimizer='AdamW', paddedSeqLen=600, parent_weights='/home/ubuntu/boda2/artifacts/my-model.epoch_5-step_19885.pkl', pretrained_weights=None, scheduler=None, scheduler_interval='epoch', scheduler_monitor=None, sequenceColumn='nt.sequence', use_batch_norm=True, use_weight_norm=False)

## Extract second-order submodule args

Get another set of submodule specific arguments based preliminary choices. (i.e., optional arguments for optimizer of choice)

In [12]:
parser = Data.add_conditional_args(parser, known_args)
parser = Model.add_conditional_args(parser, known_args)
parser = Graph.add_conditional_args(parser, known_args)

parser = Trainer.add_argparse_args(parser)
parser.add_argument('--help', '-h', action='help')
args = parser.parse_args(
    cmd_str.rstrip().split()
)

args = boda.common.utils.organize_args(parser, args)



In [13]:
args

{'positional arguments': Namespace(),
 'optional arguments': Namespace(help=None),
 'Main args': Namespace(artifact_path='gs://haddath/sgosai/deposit_test', data_module='BODA2_DataModule', graph_module='CNNTransferLearning', model_module='Basset', pretrained_weights=None),
 'Data Module args': Namespace(TestSize_pct=5.0, ValSize_pct=5.0, activityColumns=['K562', 'HepG2', 'SKNSH'], batchSize=4096, dataFile_path='../../data/BODA.MPRA.txt', numWorkers=1, paddedSeqLen=600, sequenceColumn='nt.sequence'),
 'Model Module args': Namespace(conv1_channels=300, conv1_kernel_size=19, conv2_channels=200, conv2_kernel_size=11, conv3_channels=200, conv3_kernel_size=7, dropout_p=0.3, linear1_channels=1000, linear2_channels=1000, loss_criterion='MSELoss', n_outputs=3, use_batch_norm=True, use_weight_norm=False),
 'Graph Module args': Namespace(frozen_epochs=120, optimizer='AdamW', parent_weights='/home/ubuntu/boda2/artifacts/my-model.epoch_5-step_19885.pkl', scheduler=None, scheduler_interval='epoch', 

# Run training
use modified `main` runner.

In [14]:
data_module, model_module, graph_module, model, trainer, args = main(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


Padding sequences and converting to one-hot tensors...
10000/27719 sequences padded and tokenized...
20000/27719 sequences padded and tokenized...


Key conv1.conv.weight successfully matched
Key conv1.conv.bias successfully matched
Key conv1.bn_layer.weight successfully matched
Key conv1.bn_layer.bias successfully matched
Key conv1.bn_layer.running_mean successfully matched
Key conv1.bn_layer.running_var successfully matched
Key conv1.bn_layer.num_batches_tracked successfully matched
Key conv2.conv.weight successfully matched
Key conv2.conv.bias successfully matched
Key conv2.bn_layer.weight successfully matched
Key conv2.bn_layer.bias successfully matched
Key conv2.bn_layer.running_mean successfully matched
Key conv2.bn_layer.running_var successfully matched
Key conv2.bn_layer.num_batches_tracked successfully matched
Key conv3.conv.weight successfully matched
Key conv3.conv.bias successfully matched
Key conv3.bn_layer.weight successfully matched
Key conv3.bn_layer.bias successfully matched
Key conv3.bn_layer.running_mean successfully matched
Key conv3.bn_layer.running_var successfully matched
Key conv3.bn_layer.num_batches_tracke

Found 4573903 parameters


Validation sanity check: 0it [00:00, ?it/s]

...


----------------------------------------------------------------------------
| Validation | arithmatic mean loss: 1.21141 | harmonic mean loss: 0.54801 |
----------------------------------------------------------------------------

starting epoch 138


Validating: 0it [00:00, ?it/s]


----------------------------------------------------------------------------
| Validation | arithmatic mean loss: 1.03044 | harmonic mean loss: 0.47878 |
----------------------------------------------------------------------------

starting epoch 139


Validating: 0it [00:00, ?it/s]


----------------------------------------------------------------------------
| Validation | arithmatic mean loss: 1.04981 | harmonic mean loss: 0.48359 |
----------------------------------------------------------------------------

